Warp Radolan data to GTIFF   
https://docs.wradlib.org/en/stable/georef.html#raster-functions  

Use this cell to create geotiff in gk3 or ll projection
change line 77 and 83 with correct coordinates and crs

In [ ]:
#For Files located in single folder
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 
from osgeo import osr


#Working dir paths must be clean otherwise you have to make list comprehension that only bin files in files list
wdir = "/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/"
out = "out"
#Change to Workingdir
os.chdir(wdir)

if not os.path.exists(out):
    os.mkdir(out)


#create the necessary Spatial Reference Objects for the RADOLAN-projection 
proj_stereo = wrl.georef.create_osr("dwd-radolan") #create osr object
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)
radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)

# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()
proj_gk3.ImportFromEPSG(31467) #dhdn3

#Bug handling
epsg_31467 = wrl.georef.epsg_to_osr(31467) 
epsg_4326 = wrl.georef.epsg_to_osr(4326) 


# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                       projection_source=proj_wgs,
                                       projection_target=proj_gk3)

#print(radolan_grid_gk)
#Get list of files inside the folder
files = os.listdir()
print(len(files)," files inside the folder")

#1 because of out is first file later do a list comprehension maybe more beautiful ^^
for file in files[1:]:
    proj_stereo = wrl.georef.create_osr("dwd-radolan") #create osr object
    proj_wgs = osr.SpatialReference()
    proj_wgs.ImportFromEPSG(4326)
    radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)
    #proj_gk3 = osr.SpatialReference()
    #proj_gk3.ImportFromEPSG(31467) #dhdn3
    radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
    #radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                      # projection_source=proj_wgs,
                                       #projection_target=proj_gk3)
    #print("Load: ",file ," data, georeferencing, reprojection and translate to Gtif...")
    data_raw = None
    meta = None
    data = None
    xy = None
    ds_gk3 = None
    data_raw, meta  = wrl.io.read_radolan_composite(file)
    data_raw = data_raw.round(decimals=3)
    data_raw = data_raw.astype('float32')
    #print(radolan_grid_ll[1])
    #print(data_raw.dtype)
    #print(np.median(data_raw))
    #flip the data and the coords up-down (from lower left to upper left corner)
    #data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_xy, 'upper')
    data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_ll, 'upper')
    #print(xy[-1])
    #print(xy)
    #print(radolan_grid_gk)
    #Create raster dataset: Geotiff with radolan projection
    #ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_stereo, nodata=-9999)
    ds_gk3 = wrl.georef.create_raster_dataset(data, xy, proj_wgs, -9999) # https://docs.wradlib.org/en/stable/generated/wradlib.georef.raster.create_raster_dataset.html
    #Reproject dataset to GK3
    #ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=proj_gk3, size=(900,900)) # align=(3052995.645672334, 6067805.270329222) spacing=1000 size=(900,900)
    #Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
    #This line is needed until fixed
    #ds.SetProjection(epsg_31467.ExportToWkt())
    #ds.SetProjection(epsg_4326.ExportToWkt())
    #write raster dataset
    filename = file[15:25] + ".tif"
    wrl.io.write_raster_dataset(wdir + out + "/" + filename, ds_gk3, "GTiff")
    #print("Save", filename, "to", wdir + out)

print('finish')



In [2]:
from osgeo import osr
import wradlib as wrl

#Data Paths
wdir = "/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/"
out2 = "test/"
file1 = "raa01-sf_10000-0901012350-dwd---bin"
file2 = "raa01-sf_10000-1907312350-dwd---bin"

proj_stereo = wrl.georef.create_osr("dwd-radolan") #(osr.SpatialReference) – GDAL/OSR object defining projection
proj_wgs = osr.SpatialReference() #Initialize a SpatialReference object
proj_wgs.ImportFromEPSG(4326) # Spatial Referenz for EPSG 3426
#print(proj_wgs)

radolan_grid_xy = wrl.georef.get_radolan_grid(900,900) 
radolan_grid_ll = wrl.georef.projection.reproject(radolan_grid_xy, projection_source=proj_stereo, projection_target=proj_wgs)
#print("{0}, ({1:.4f}, {2:.4f})".format(radolan_grid_ll.shape, *radolan_grid_ll[0,0,:]))

# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()  #Initialize a SpatialReference object
proj_gk3.ImportFromEPSG(31467) #Spatial Referenz for EPSG 31467
#print(proj_gk3)

# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,projection_source=proj_wgs,projection_target=proj_gk3)
#print("{0}, ({1:.4f}, {2:.4f})".format(radolan_grid_gk.shape, *radolan_grid_gk[0,0,:]))

data_raw, attributes = wrl.io.radolan.read_radolan_composite(wdir + file1, missing=-9999, loaddata=True) #data : numpy.array of shape (number of rows, number of columns) | xarray.Dataset >// attrs : dictionary of metadata information from the file heade
data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_xy, 'upper')

rad_stereo = wrl.georef.create_raster_dataset(data, xy, projection=proj_stereo, nodata=-9999) #dataset (gdal.Dataset) – In-Memory raster dataset

cols = rad_stereo.RasterXSize
rows = rad_stereo.RasterYSize

filename = file1[15:25] + "_stereo" + ".tif"

wrl.io.write_raster_dataset(wdir + out2 + "/" + filename, rad_stereo, "GTiff")

rad_wgs =  wrl.georef.raster.reproject_raster_dataset(rad_stereo, size = (cols,rows), projection_target = proj_wgs) 

filename = file1[15:25] + "_wgs_tr + out2 + "/" + filename, rad_wgs, "GTiff")

rad_gk3 =  wrl.georef.raster.reproject_raster_dataset(rad_wgs, size = (cols,rows), projection_target = proj_gk3) 

filename = file1[15:25] + "_gk3_1" + ".tif"

wrl.io.write_raster_dataset(wdir + out2 + "/" + filename, rad_gk3, "GTiff")




SyntaxError: invalid syntax (<ipython-input-2-5f3b253fd6a5>, line 42)

In [8]:
from osgeo import osr
import wradlib as wrl
import os

#Data Paths
wdir = "/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/"
out2 = "out/"
os.chdir(wdir)
files = os.listdir()
files = files[1:]

proj_stereo = wrl.georef.create_osr("dwd-radolan") #(osr.SpatialReference) – GDAL/OSR object defining projection
proj_wgs = osr.SpatialReference() #Initialize a SpatialReference object
proj_wgs.ImportFromEPSG(4326) # Spatial Referenz for EPSG 3426
#print(proj_wgs)

radolan_grid_xy = wrl.georef.get_radolan_grid(900,900) 
radolan_grid_ll = wrl.georef.projection.reproject(radolan_grid_xy, projection_source=proj_stereo, projection_target=proj_wgs)
#print("{0}, ({1:.4f}, {2:.4f})".format(radolan_grid_ll.shape, *radolan_grid_ll[0,0,:]))

# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()  #Initialize a SpatialReference object
proj_gk3.ImportFromEPSG(31467) #Spatial Referenz for EPSG 31467
#print(proj_gk3)

# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,projection_source=proj_wgs,projection_target=proj_gk3)
#print("{0}, ({1:.4f}, {2:.4f})".format(radolan_grid_gk.shape, *radolan_grid_gk[0,0,:]))

amount = len(files)
print(amount)
n=0

for file in files:
    n=n+1
    data_raw, attributes = wrl.io.radolan.read_radolan_composite(wdir + file, missing=-9999, loaddata=True) #data : numpy.array of shape (number of rows, number of columns) | xarray.Dataset >// attrs : dictionary of metadata information from the file heade
    data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_xy, 'upper')

    rad_stereo = wrl.georef.create_raster_dataset(data, xy, projection=proj_stereo, nodata=-9999) #dataset (gdal.Dataset) – In-Memory raster dataset

    cols = rad_stereo.RasterXSize
    rows = rad_stereo.RasterYSize

    filename = file[15:25] + ".tif"

    wrl.io.write_raster_dataset(wdir + out2 + "/" + filename, rad_stereo, "GTiff")
    print(str(n)+"/"+ str(amount))



3862
1/3862
2/3862
3/3862
4/3862
5/3862
6/3862
7/3862
8/3862
9/3862
10/3862
11/3862
12/3862
13/3862
14/3862
15/3862
16/3862
17/3862
18/3862
19/3862
20/3862
21/3862
22/3862
23/3862
24/3862
25/3862
26/3862
27/3862
28/3862
29/3862
30/3862
31/3862
32/3862
33/3862
34/3862
35/3862
36/3862
37/3862
38/3862
39/3862
40/3862
41/3862
42/3862
43/3862
44/3862
45/3862
46/3862
47/3862
48/3862
49/3862
50/3862
51/3862
52/3862
53/3862
54/3862
55/3862
56/3862
57/3862
58/3862
59/3862
60/3862
61/3862
62/3862
63/3862
64/3862
65/3862
66/3862
67/3862
68/3862
69/3862
70/3862
71/3862
72/3862
73/3862
74/3862
75/3862
76/3862
77/3862
78/3862
79/3862
80/3862
81/3862
82/3862
83/3862
84/3862
85/3862
86/3862
87/3862
88/3862
89/3862
90/3862
91/3862
92/3862
93/3862
94/3862
95/3862
96/3862
97/3862
98/3862
99/3862
100/3862
101/3862
102/3862
103/3862
104/3862
105/3862
106/3862
107/3862
108/3862
109/3862
110/3862
111/3862
112/3862
113/3862
114/3862
115/3862
116/3862
117/3862
118/3862
119/3862
120/3862
121/3862
122/3862
123/3

923/3862
924/3862
925/3862
926/3862
927/3862
928/3862
929/3862
930/3862
931/3862
932/3862
933/3862
934/3862
935/3862
936/3862
937/3862
938/3862
939/3862
940/3862
941/3862
942/3862
943/3862
944/3862
945/3862
946/3862
947/3862
948/3862
949/3862
950/3862
951/3862
952/3862
953/3862
954/3862
955/3862
956/3862
957/3862
958/3862
959/3862
960/3862
961/3862
962/3862
963/3862
964/3862
965/3862
966/3862
967/3862
968/3862
969/3862
970/3862
971/3862
972/3862
973/3862
974/3862
975/3862
976/3862
977/3862
978/3862
979/3862
980/3862
981/3862
982/3862
983/3862
984/3862
985/3862
986/3862
987/3862
988/3862
989/3862
990/3862
991/3862
992/3862
993/3862
994/3862
995/3862
996/3862
997/3862
998/3862
999/3862
1000/3862
1001/3862
1002/3862
1003/3862
1004/3862
1005/3862
1006/3862
1007/3862
1008/3862
1009/3862
1010/3862
1011/3862
1012/3862
1013/3862
1014/3862
1015/3862
1016/3862
1017/3862
1018/3862
1019/3862
1020/3862
1021/3862
1022/3862
1023/3862
1024/3862
1025/3862
1026/3862
1027/3862
1028/3862
1029/3862
1030/38

1750/3862
1751/3862
1752/3862
1753/3862
1754/3862
1755/3862
1756/3862
1757/3862
1758/3862
1759/3862
1760/3862
1761/3862
1762/3862
1763/3862
1764/3862
1765/3862
1766/3862
1767/3862
1768/3862
1769/3862
1770/3862
1771/3862
1772/3862
1773/3862
1774/3862
1775/3862
1776/3862
1777/3862
1778/3862
1779/3862
1780/3862
1781/3862
1782/3862
1783/3862
1784/3862
1785/3862
1786/3862
1787/3862
1788/3862
1789/3862
1790/3862
1791/3862
1792/3862
1793/3862
1794/3862
1795/3862
1796/3862
1797/3862
1798/3862
1799/3862
1800/3862
1801/3862
1802/3862
1803/3862
1804/3862
1805/3862
1806/3862
1807/3862
1808/3862
1809/3862
1810/3862
1811/3862
1812/3862
1813/3862
1814/3862
1815/3862
1816/3862
1817/3862
1818/3862
1819/3862
1820/3862
1821/3862
1822/3862
1823/3862
1824/3862
1825/3862
1826/3862
1827/3862
1828/3862
1829/3862
1830/3862
1831/3862
1832/3862
1833/3862
1834/3862
1835/3862
1836/3862
1837/3862
1838/3862
1839/3862
1840/3862
1841/3862
1842/3862
1843/3862
1844/3862
1845/3862
1846/3862
1847/3862
1848/3862
1849/3862


2570/3862
2571/3862
2572/3862
2573/3862
2574/3862
2575/3862
2576/3862
2577/3862
2578/3862
2579/3862
2580/3862
2581/3862
2582/3862
2583/3862
2584/3862
2585/3862
2586/3862
2587/3862
2588/3862
2589/3862
2590/3862
2591/3862
2592/3862
2593/3862
2594/3862
2595/3862
2596/3862
2597/3862
2598/3862
2599/3862
2600/3862
2601/3862
2602/3862
2603/3862
2604/3862
2605/3862
2606/3862
2607/3862
2608/3862
2609/3862
2610/3862
2611/3862
2612/3862
2613/3862
2614/3862
2615/3862
2616/3862
2617/3862
2618/3862
2619/3862
2620/3862
2621/3862
2622/3862
2623/3862
2624/3862
2625/3862
2626/3862
2627/3862
2628/3862
2629/3862
2630/3862
2631/3862
2632/3862
2633/3862
2634/3862
2635/3862
2636/3862
2637/3862
2638/3862
2639/3862
2640/3862
2641/3862
2642/3862
2643/3862
2644/3862
2645/3862
2646/3862
2647/3862
2648/3862
2649/3862
2650/3862
2651/3862
2652/3862
2653/3862
2654/3862
2655/3862
2656/3862
2657/3862
2658/3862
2659/3862
2660/3862
2661/3862
2662/3862
2663/3862
2664/3862
2665/3862
2666/3862
2667/3862
2668/3862
2669/3862


3391/3862
3392/3862
3393/3862
3394/3862
3395/3862
3396/3862
3397/3862
3398/3862
3399/3862
3400/3862
3401/3862
3402/3862
3403/3862
3404/3862
3405/3862
3406/3862
3407/3862
3408/3862
3409/3862
3410/3862
3411/3862
3412/3862
3413/3862
3414/3862
3415/3862
3416/3862
3417/3862
3418/3862
3419/3862
3420/3862
3421/3862
3422/3862
3423/3862
3424/3862
3425/3862
3426/3862
3427/3862
3428/3862
3429/3862
3430/3862
3431/3862
3432/3862
3433/3862
3434/3862
3435/3862
3436/3862
3437/3862
3438/3862
3439/3862
3440/3862
3441/3862
3442/3862
3443/3862
3444/3862
3445/3862
3446/3862
3447/3862
3448/3862
3449/3862
3450/3862
3451/3862
3452/3862
3453/3862
3454/3862
3455/3862
3456/3862
3457/3862
3458/3862
3459/3862
3460/3862
3461/3862
3462/3862
3463/3862
3464/3862
3465/3862
3466/3862
3467/3862
3468/3862
3469/3862
3470/3862
3471/3862
3472/3862
3473/3862
3474/3862
3475/3862
3476/3862
3477/3862
3478/3862
3479/3862
3480/3862
3481/3862
3482/3862
3483/3862
3484/3862
3485/3862
3486/3862
3487/3862
3488/3862
3489/3862
3490/3862


In [ ]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Working dir
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/"
file = "raa01-sf_10000-1701012350-dwd---bin"
#Read the original data: data = 2d Array(row,col) and metadata (productType, datetime,nodataflag)
#Origin in lower left corner
data_raw, meta  = wrl.io.read_radolan_composite(wdir + file)

#Round data to 2 decimal places to save memory
data_raw_s = np.around(data_raw, decimals = 2, out = None)

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)

#flip the data and the coords up-down (from lower left to upper left corner)
data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')

#Create raster dataset: Geotiff with radolan projection
ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)

#Reproject dataset to EPSG 3857
ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
#Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
#This line is needed until fixed
ds2.SetProjection(epsg_3857.ExportToWkt())

#write raster dataset
filename = file[15:25] + ".tif"
wrl.io.write_raster_dataset(wdir + filename, ds2, "GTiff")
#

In [ ]:
from osgeo import gdal
import numpy as np
raster = None
array = None
raster = gdal.Open("/media/elisabeth/0A581C5C581C4939/master/radolan_data/out/bin/2009-2019/out/0901022350.tif")
#band = raster.GetRasterBand(1)
array = raster.ReadAsArray()
print(np.median(array))


In [ ]:
# load radolan files
print(os.getcwd())
rw_filename = wrl.util.get_wradlib_data_file("raa01-sf_10000-0901012350-dwd---bin.gz")
ds, rwattrs = wrl.io.read_radolan_composite(rw_filename, loaddata='xarray')
# print the xarray dataset
print(ds)